In [1]:
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

import argparse

import numpy as np
import tensorflow as tf

In [2]:
def load_graph(model_file):
  graph = tf.Graph()
  graph_def = tf.compat.v1.GraphDef()

  with open(model_file, "rb") as f:
    graph_def.ParseFromString(f.read())
  with graph.as_default():
    tf.import_graph_def(graph_def)

  return graph

In [3]:
def read_tensor_from_image_file(file_name,
                                input_height=299,
                                input_width=299,
                                input_mean=0,
                                input_std=255):
  input_name = "file_reader"
  output_name = "normalized"
  with tf.compat.v1.Session() as ses:
    file_reader = tf.io.read_file(file_name, input_name)
    if file_name.endswith(".png"):
      image_reader = tf.io.decode_png(file_reader, channels=3, name="png_reader")
    elif file_name.endswith(".gif"):
      image_reader = tf.squeeze(tf.io.decode_gif(file_reader, name="gif_reader"))
    elif file_name.endswith(".bmp"):
      image_reader = tf.io.decode_bmp(file_reader, name="bmp_reader")
    else:
      image_reader = tf.io.decode_jpeg(
          file_reader, channels=3, name="jpeg_reader")
    float_caster = tf.cast(image_reader, tf.float32)
    dims_expander = tf.expand_dims(float_caster, 0)
    resized = tf.compat.v1.image.resize_bilinear(dims_expander, [input_height, input_width])
    normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
    #sess = tf.compat.v1.Session()
    return ses.run(normalized)


In [4]:
def load_labels(label_file):
  proto_as_ascii_lines = tf.io.gfile.GFile(label_file).readlines()
  return [l.rstrip() for l in proto_as_ascii_lines]

In [5]:
if __name__ == "__main__":
  file_name = "tensorflow/examples/label_image/data/grace_hopper.jpg"
  model_file = \
    "tensorflow/examples/label_image/data/inception_v3_2016_08_28_frozen.pb"
  label_file = "tensorflow/examples/label_image/data/imagenet_slim_labels.txt"
  input_height = 299
  input_width = 299
  input_mean = 0
  input_std = 255
  input_layer = "input"
  output_layer = "InceptionV3/Predictions/Reshape_1"

In [6]:
class Args():
    image='D:\Drone_project\Test_folder'
    graph='D:\Drone_project/Test1IV3/retrained_graph.pb'
    labels='D:\Drone_project/Test1IV3/retrained_labels.txt'
    input_layer='Placeholder'
    output_layer='final_result'
    input_height=299
    input_width=299
    input_mean=0
    input_std=255

args=Args()

In [7]:
  if args.graph:
    model_file = args.graph
  if args.image:
    file_name = args.image
  if args.labels:
    label_file = args.labels
  if args.input_height:
    input_height = args.input_height
  if args.input_width:
    input_width = args.input_width
  if args.input_mean:
    input_mean = args.input_mean
  if args.input_std:
    input_std = args.input_std
  if args.input_layer:
    input_layer = args.input_layer
  if args.output_layer:
    output_layer = args.output_layer

In [8]:
import os
import shutil
filepaths = []
for list_of_images  in os.listdir(file_name):
    filepaths.append(list_of_images)
    
print(filepaths)

['0001test.png', '0002test.png', '0003test.png', '0004test.png', '0005test.png', '0006test.png', '0007test.png', '0008test.png', '0009test.png', '0010test.png', '0011test.png', '0012test.png', '0013test.png', '0014test.png', '0015test.png', '0016test.png', '0017test.png', '0018test.png', '0019test.png', '0020test.png', '0021test.png', '0022test.png', '0023test.png', '0024test.png', '0025test.png', '0026test.png', '0027test.png', '0028test.png', '0029test.png', '0030test.png', '0031test.png', '0032test.png', '0033test.png', '0034test.png', '0035test.png', '0036test.png', '0037test.png', '0038test.png', '0039test.png', '0040test.png', '0041test.png', '0042test.png', '0043test.png', '0044test.png', '0045test.png', '0046test.png', '0047test.png', '0048test.png', '0049test.png', '0050test.png', '0051test.png', '0052test.png', '0053test.png', '0054test.png', '0055test.png', '0056test.png', '0057test.png', '0058test.png', '0059test.png', '0060test.png', '0061test.png', '0062test.png', '0063te

In [9]:
graph = load_graph(model_file)

In [10]:
print(filepaths[2])

0003test.png


In [11]:
  sum = 0
  cluster1 = 0
  cluster1count = 0
  cluster2 = 0
  cluster2count = 0  
  cluster3 = 0
  cluster3count = 0
  cluster4 = 0
  cluster4count = 0
  for images in os.listdir(file_name):
    print(images)
    image_path = r"D:\Drone_project\Test_folder\{}".format(images)
    t = read_tensor_from_image_file(
      image_path,
      input_height=input_height,
      input_width=input_width,
      input_mean=input_mean,
      input_std=input_std)
    input_name = "import/" + input_layer
    output_name = "import/" + output_layer
    input_operation = graph.get_operation_by_name(input_name)
    output_operation = graph.get_operation_by_name(output_name)
  
    with tf.compat.v1.Session(graph=graph) as sess:
      results = sess.run(output_operation.outputs[0], {
        input_operation.outputs[0]: t
    })
    results = np.squeeze(results)
    top_k = results.argsort()[-5:][::-1]
    labels = load_labels(label_file)
    top = top_k[0]
    print(labels[top], results[top]) 
    sum = sum + results[top]
    if top_k[0] == 0:
       cluster1 = cluster1 + results[top]
       cluster1count = cluster1count + 1
    if top_k[0] == 1:
       cluster2 = cluster2 + results[top]
       cluster2count = cluster2count + 1
    if top_k[0] == 2:
       cluster3 = cluster3 + results[top]
       cluster3count = cluster3count + 1
    if top_k[0] == 3:
       cluster4 = cluster4 + results[top]
       cluster4count = cluster4count + 1
    
    #for i in top_k:
     #print(labels[i], results[i])

0001test.png
1 0.75044274
0002test.png
2 0.9987704
0003test.png
2 0.940653
0004test.png
2 0.9994286
0005test.png
2 0.99250185
0006test.png
2 0.9982033
0007test.png
2 0.98315424
0008test.png
2 0.996825
0009test.png
2 0.9974075
0010test.png
0 0.73095757
0011test.png
2 0.48680896
0012test.png
2 0.68777597
0013test.png
2 0.96293473
0014test.png
3 0.75945324
0015test.png
1 0.93161315
0016test.png
1 0.99075806
0017test.png
1 0.97871155
0018test.png
0 0.99437666
0019test.png
2 0.9890969
0020test.png
1 0.4225789
0021test.png
2 0.70476997
0022test.png
2 0.9761622
0023test.png
2 0.97686505
0024test.png
3 0.7467302
0025test.png
3 0.66807395
0026test.png
2 0.99358165
0027test.png
0 0.5622567
0028test.png
3 0.99834
0029test.png
0 0.960728
0030test.png
2 0.91565835
0031test.png
3 0.7931077
0032test.png
0 0.94457155
0033test.png
0 0.99015826
0034test.png
0 0.3626021
0035test.png
1 0.7831832
0036test.png
1 0.7071996
0037test.png
1 0.56122476
0038test.png
0 0.74837583
0039test.png
3 0.91105723
0040test

In [24]:
 average = sum/231
 print("Average", average)
 print("1: ", cluster1/cluster1count)
 print("2: ", cluster2/cluster2count)
 print("3: ", cluster3/cluster3count)
 print("4: ", cluster4/cluster4count)

Average 0.835282456952256
1:  0.8277356956686293
2:  0.8179443409809699
3:  0.8140964765174716
4:  0.8785641866511312


In [27]:
  graph = load_graph(model_file)
  t = read_tensor_from_image_file(
      file_name,
      input_height=input_height,
      input_width=input_width,
      input_mean=input_mean,
      input_std=input_std)

In [28]:
  input_name = "import/" + input_layer
  output_name = "import/" + output_layer
  input_operation = graph.get_operation_by_name(input_name)
  output_operation = graph.get_operation_by_name(output_name)

In [29]:
  with tf.compat.v1.Session(graph=graph) as sess:
    results = sess.run(output_operation.outputs[0], {
        input_operation.outputs[0]: t
    })
  results = np.squeeze(results)

In [30]:
  top_k = results.argsort()[-5:][::-1]
  labels = load_labels(label_file)
  for i in top_k:
    print(labels[i], results[i])

2 0.9777003
3 0.01622103
1 0.0026042594
4 0.0024725639
0 0.0010019171


In [17]:
  parser = argparse.ArgumentParser()
  parser.add_argument("--image", help="image to be processed")
  parser.add_argument("--graph", help="graph/model to be executed")
  parser.add_argument("--labels", help="name of file containing labels")
  parser.add_argument("--input_height", type=int, help="input height")
  parser.add_argument("--input_width", type=int, help="input width")
  parser.add_argument("--input_mean", type=int, help="input mean")
  parser.add_argument("--input_std", type=int, help="input std")
  parser.add_argument("--input_layer", help="name of input layer")
  parser.add_argument("--output_layer", help="name of output layer")
  args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--image IMAGE] [--graph GRAPH] [--labels LABELS] [--input_height INPUT_HEIGHT]
                             [--input_width INPUT_WIDTH] [--input_mean INPUT_MEAN] [--input_std INPUT_STD]
                             [--input_layer INPUT_LAYER] [--output_layer OUTPUT_LAYER]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Dennis\AppData\Roaming\jupyter\runtime\kernel-8925a7c7-f4ca-427a-8900-19d81abbd0a8.json


SystemExit: 2